# Lecture 05: Frank-Wolfe Algorithm - Foundations

```{note}
This lecture introduces the foundations of the Frank-Wolfe Algorithm - a gradient descent algorithm, to solve the Traffic Assignment Problem. 
```

---

## Introduction 

The Frank–Wolfe algorithm, is a classical gradient descent algorithm for solving smooth convex optimization problems over a compact convex set. It was first introduced in 1956 by Marguerite Frank and Philip Wolfe in their seminal paper “An algorithm for quadratic programming and applications.” Since then, it has become a cornerstone method in optimization theory and has found wide applications in areas such as traffic assignment, machine learning, structural optimization, and signal processing.

For a convex and continuously differentiable function $f(\mathbf{x})$ with a compact and convext feasible region $\mathcal{D}\subset\mathbb{R}^n$, the optimization algorithm proceeds iteratively by:

1. **Linearization:**  $f(\mathbf{x}) = f(\mathbf{x_o}) + \nabla f(\mathbf{x_o})^\top(\mathbf{x}-\mathbf{x_o})$, thereby transforming a non-linear function into a linear approximation

    $$
    \min_{\mathbf{y}\in\mathcal{D}} \ \nabla f(\mathbf{x})^\top(\mathbf{y}-\mathbf{x}).
    $$
    Here, $\mathbf{y}$ is the auxiliary point, i.e., the optimal solution of the linearized problem.

2. **Direction of descent:** $\mathbf{d} = \mathbf{y} - \mathbf{x}$.

3. **Step-size (line search):** choose $\lambda \in [0,1]$ (exact or inexact line search), e.g.,

   $$
   \lambda \gets \arg\min_{\lambda\in[0,1]} f(\mathbf{x} + \lambda\,\mathbf{d}).
   $$

4. **Update:** 

   $$
   \mathbf{x} = \mathbf{x} + \lambda \mathbf{d},
   $$

   which remains feasible because it is a convex combination of feasible points.

### Frank-Wolfe Algorithm

1. **Procedure** $\text{FW}(G = (N, R, S, A, Q, C))$
3. $X \gets \{0 \ \forall \ a \in A\}$ &emsp;<small>// initialize zero flow on all arcs</small>
4. $\text{converged} \gets false$
5. **while not** $\text{converged}$ **do** &emsp;<small>// iterate until converged</small>
6. &emsp; $C \gets \{c_a(x_a) \ \forall \ a \in A\}$ &emsp;<small>// compute arc costs</small>
7. &emsp; $Y \gets \text{AoN}(G)$ &emsp;<small>// fetch auxilary arc flows through All-or-Nothing Assignment</small>
8. &emsp; $D \gets Y - X$ &emsp;<small>// evaluate descent direction</small>
9. &emsp; $\lambda^* \gets \argmin_{\lambda} \sum_a \int^{x_a + \lambda d_a}_0 c_a(x)dx$ &emsp;<small>// compute optimal step size in the direction of descent</small>
10. &emsp; $X \gets X + \lambda^* D$ &emsp;<small>// update arc flows</small>
11. &emsp; **if** $\sqrt{\sum_a (\lambda^* d_a)^2} / \sum_a x_a \leq \epsilon$ **then** &emsp;<small>// check for convergence</small>
12. &emsp;&emsp; $\text{converged} \gets true$
13. &emsp; **end if**
14. **end while**
15. **return** $X$ &emsp;<small>// return arc flows</small>

### All-or-Nothing Assignment

1. **Procedure** $\text{AoN}(G = (N, R, S, A, Q, C))$
2. **for** $r \in R$ **do** &emsp;<small>// loop over all origin nodes</small>
3. &emsp; $L \gets \text{label}(r, G)$ &emsp;<small>// establish predecessor labels for each node from the origin node</small>
4. &emsp; **for** $s \in S$ **do** &emsp;<small>// loop over all destination nodes</small>
5. &emsp;&emsp; **for** $a \in \text{path}(r, s, L)$ &emsp;<small>// estbalish the least cost path</small>
6. &emsp;&emsp;&emsp; $x_a \gets x_a + q_{rs}$ &emsp;<small>// dump all flows onto this path (All-or-Nothing)</small>
7. &emsp;&emsp; **end for**
8. &emsp; **end for**
9. $X \gets \{x_a; \ \forall \ a \in A\}$ &emsp;<small>// return arc flows</small>
10. **return** $X$

### Labeling Algorithm

1. **Procedure** $\text{label}(r, G = (N, R, S, A, Q, C))$
2. $X ← \{n; n \in N\}$ &emsp;<small>// initialize a set of open nodes</small>
3. $L ← \{0; n \in N\}$ &emsp;<small>// initialize predecessor label for each node </small>
4. $Z ← \{\infty; n \in N\}$ &emsp;<small>// initialize cost label for each node</small>
5. $i ← r$ &emsp;<small>// set pivot node to origin node</small>
6. $P_i ← r$ &emsp;<small>// update predecessor label</small>
7. $Z_i ← 0$ &emsp;<small>// update cost label</small>
8. $X ← X \backslash \{i\}$ &emsp;<small>// remove the pivot node from the set of open nodes</small>
9. **while** $X \ne \phi $ **do** &emsp;<small>// iterate until all nodes have been visited</small>
10. &emsp; **for** $j \in H(i)$ **do** &emsp;<small>// loop over all successor nodes of the pivot nodes</small>
11. &emsp;&emsp; $z ← Z_i + C_{ij}$ &emsp;<small>// evaluate traversal cost from the pivot node to the head node</small>
12. &emsp;&emsp; **if** $z < Z_j$ **then** &emsp;<small>// compare costs</small>
13. &emsp;&emsp;&emsp; $L_j ← i$ &emsp;<small>// update predecessor label</small>
14. &emsp;&emsp;&emsp; $Z_j ← z$ &emsp;<small>// update cost label</small>
15. &emsp;&emsp; **end if**
16. &emsp; **end for**
17. &emsp; $i ← \text{argmin}\{Z_k; k \in X\}$ &emsp;<small>// set pivot node to an open node with least cost label</small>
18. &emsp; $X ← X \backslash \{i\}$ &emsp;<small>// remove the pivot node from the set of open nodes</small>
19. **end while**
20. **return** $L$ &emsp;<small>// return predecessor labels</small>

### Path Algorithm

1. **Procedure** $\text{path}(r, s, L)$
2. $P ← \phi$ &emsp;<small>// initialize path vector</small>
3. $t ← s$ &emsp;<small>// set tail node to destination</small>
4. $h ← s$ &emsp;<small>// set head node to destination</small>
5. **while** $h \ne r$ **do**
6. &emsp; $t ← L_h$ &emsp;<small>// set tail node to the predecessor of head node</small>
7. &emsp; $P ← P \cup \{(t,h)\}$ &emsp;<small>// add arc to path</small>
8. &emsp; $h ← t$&emsp;<small>// add pivot node to path</small>
9. **end while**
10. $P ← \text{reverse}(P)$ &emsp;<small>// reverse path vector</small>
11. **return** $P$ &emsp;<small>// return path vector</small>

---

## Example

Solve User Equilibrium for the following two example networks using the Frank-Wolfe method.

### Network I

<p align="center">
  <img src="https://raw.githubusercontent.com/anmpahwa/CE5810/refs/heads/main/resources/TAPNetwork-I.png" width="40%"/>
</p>
<p align="center">
  <b>Figure 1.</b> Transportation Network - I
</p>

### Network II

<p align="center">
  <img src="https://raw.githubusercontent.com/anmpahwa/CE5810/refs/heads/main/resources/TAPNetwork-II.png" width="66%"/>
</p>
<p align="center">
  <b>Figure 2.</b> Transportation Network - II
</p>

---

```{note}
In the next lecture, we will implement the Frank-Wolfe algorithm on some real-world transportation networks such as Sioux Falls, Anaheim, and Los Angeles, among others.
```
